In [1]:
import os
import pandas as pd
import pickle
import json
from Bio import SeqIO
from Bio import AlignIO   #, Align


%load_ext autoreload
%autoreload 2
import diverse_yeast_tools as dyt

base_dir = os.path.normpath('G:/My Drive/Crick_LMS/projects/diverse_yeasts/alphafold')
divyeast_dir = os.path.normpath('C:/Users/heineib/Documents/GitHub/diverse_yeast')
y1000plus_dir = os.path.normpath('C:/Users/heineib/Documents/GitHub/y1000plus_tools/data') + os.sep
genomes_dir = os.path.normpath('G:/My Drive/Crick_LMS/external_data/genomes')


In [2]:
#Load main analysis file
struct_analysis = pickle.load(open(base_dir + os.sep + os.path.normpath('Output/data/Filter_clusters.pkl'), 'rb'))

#Filter_clusters was made using Nir's clustering data.  
# Oliver said "we have to drop C0 in Nir's data. It affects OG1380 Q07732 and OG4312 P38280 in S.cer (REF)"
#The old analysis file had slightly different clusters to drop. 
#struct_analysis = pickle.load(open(base_dir + os.sep + os.path.normpath('Output/data/Analysis_new_02.pkl'), 'rb'))
#OG1004__Scer_AF-P20095-F1-model_v2 and OG1004__Scer_AF-P15938-F1-model_v2 are examples where the orthogroup divides into smaller groups. 

#Load original sequence file, and make dictionary of gene_id to fasta header and gene_id to peptide sequence
#Read in Sequence File, extract all names make dict of seq_alignment sequences
selected_proteins = SeqIO.parse(base_dir +os.sep +  'selected_proteins.fasta', 'fasta')
selected_proteins_headers = {}
selected_proteins_seqs = {}
for record in selected_proteins: 
    selected_proteins_headers[record.id] = record.description
    selected_proteins_seqs[record.id] = str(record.seq)

#Load peptide sequences for model species, make dictionary from gene id to peptide sequence
model_protein_dict = {}
for spec_abbrev in ['Scer', 'Spom', 'Calb']: 
    model_protein_dict[spec_abbrev] = dyt.load_model_protein_dict(spec_abbrev)
    

#ID Mapping for model species
#Lookup from gene_id to y1000_id 
gene_id_2_y1000_id = dyt.load_model_gene_id_2_y1000_id()

#swissprot_id_2_gene_id = dyt.load_model_swissprot_id_2_gene_id()
swissprot_id_2_gene_id = dyt.load_model_swissprot_id_2_gene_id()


In [9]:
a = list(struct_analysis.keys())[0]
a = struct_analysis[a]['Files to be included']

['Calb_AF-A0A1D8PMC5-F1-model_v2',
 'REF_Scer_AF-P38840-F1-model_v2',
 'Spom_AF-O14192-F1-model_v2',
 'alloascoidea_hylecoeti__OG3639__0_2924',
 'ascoidea_rubescens__OG3639__6_326',
 'ascoidea_rubescens__OG3639__6_5407',
 'candida_tropicalis__OG3639__30_2724',
 'cyberlindnera_jadinii__OG3639__35_5353',
 'cyberlindnera_jadinii__OG3639__35_5427',
 'kazachstania_naganishii__OG3639__49_4799',
 'kluyveromyces_lactis__OG3639__50_1047',
 'kluyveromyces_marxianus__OG3639__51_1738',
 'komagataella_pastoris__OG3639__52_2093',
 'lachancea_thermotolerans__OG3639__64_114',
 'ogataea_parapolymorpha__OG3639__104_1542',
 'pachysolen_tannophilus__OG3639__106_4781',
 'torulaspora_delbrueckii__OG3639__135_1978',
 'vanderwaltozyma_polyspora__OG3639__136_4279',
 'wickerhamomyces_anomalus__OG3639__139_4915',
 'wickerhamomyces_anomalus__OG3639__139_5127',
 'yHMPu5000034957_hanseniaspora_osmophila_160519__OG3639__247_2276',
 'yarrowia_lipolytica__OG3639__144_5496',
 'zygosaccharomyces_rouxii__OG3639__342_3711

In [25]:
#Filter original structural alignments to remove bad structures
#Also include metadata from original species selection fasta and add metadata for model species. 
# base_dir/selected_proteins.fasta
#
#
# Make a protein fasta for each alignment from orignal sequence and with model species sequences
#
# Make a dictionary of lists of each protein present in each species - save as a .json
# This json is keyed on species and has elements
#[name_orig,gene_id,y1000_id,swissprot_id,source]

species_table = pd.read_csv(base_dir + os.sep + 'species_selection.csv')
specs = list(species_table.loc[species_table['Load']=='Y']['original_genome_id'])
proteins_present_by_spec = {spec: [] for spec in specs}

modelspec_params = {'Scer':('saccharomyces_cerevisiae'), 
                    'Calb':('candida_albicans'), 
                    'Spom':('schizosaccharomyces_pombe')
                   }
proteins_present_by_spec_fname = base_dir + os.sep + 'selected_protein_ids.json'


for fasta_fname in os.listdir(base_dir + os.sep +  os.path.normpath('msas\structural\FASTA') + os.sep  ):
    #fasta_fname = 'OG1004_REF_Scer_AF-P15938-F1-model_v2.FASTA'
    fasta_fname_base = fasta_fname.split('.')[0]

    seqs_to_remove = struct_analysis[fasta_fname.split('.')[0]]['Files to be excluded']

    fname_struct_aln_orig = base_dir + os.sep +  os.path.normpath('msas\structural\FASTA\\' + fasta_fname)   # os.path.normpath('msas\FILES_ogs_pep_aligned\\' + og + '.mfaa.mafft')
    fname_struct_aln_filt_out = base_dir + os.sep + os.path.normpath('msas\structural\\fasta_filt\\' + fasta_fname_base + '.struct_filt.fasta')
    fname_proteome = base_dir + os.sep + os.path.normpath('og_sequences\proteome\\' + fasta_fname_base + '.pep.fasta')

    #Read in dictionary of ref name map to sequence name
    struct_align = AlignIO.read(fname_struct_aln_orig, 'fasta')
    with open(fname_struct_aln_filt_out , 'w') as f_out_filt:
        with open(fname_proteome, 'w') as f_out_prot: 
            for record in struct_align: 
                structure_imported = False
                name_orig = record.id    

                #Check if name in ref sequence (S.cer, C.alb, and S. pom) + convert
                if name_orig.split('_')[0] in set(['Scer', 'REF', 'Calb', 'Spom']):
                    structure_imported = True
                    if name_orig.split('_')[0] == 'REF': 
                        spec_abbrev = name_orig.split('_')[1]
                        swissprot_id = name_orig.split('_')[2].split('-')[1]
                    else: 
                        spec_abbrev = name_orig.split('_')[0]
                        swissprot_id = name_orig.split('_')[1].split('-')[1]
                    
                    (spec) = modelspec_params[spec_abbrev]
                    gene_id = swissprot_id_2_gene_id[spec_abbrev][swissprot_id]
                    if spec_abbrev == 'Spom':
                        y1000_id = 'None'
                    else: 
                        #spec_old = spec_abbrev_dict[spec_abbrev]
                        y1000_id = gene_id_2_y1000_id[spec_abbrev][gene_id]

                else: 
                    (spec, og, y1000_id) = name_orig.split('__')


                #Filter out seqs with bad alignments
                if not(name_orig in seqs_to_remove): 
                    
                    #Extract sequence from original peptide fasta
                    
                    if structure_imported: 
                        prot_seq = model_protein_dict[spec_abbrev][gene_id]
                        L = len(prot_seq)
                        header = '>' + name_orig + ' source=af2  gene_full=' + gene_id +' y1000_id=' + y1000_id + ' L=' + str(L) + '\n' 
                        
                        proteins_present_by_spec[spec].append((name_orig,gene_id,y1000_id,swissprot_id,'af2'))
                    
                        
                    else: 
                        header_dict = {}
                        header_raw =selected_proteins_headers[name_orig] 
                        for item in header_raw.split(' ')[1:]: 
                            key,val = item.split('=')
                            header_dict[key] = val
                        
                        source = header_dict['source']
                        if source=='uniprot':
                            swissprot_id = header_dict['gene_full'].split('|')[1]
                        elif source=='shen':
                            swissprot_id = None

                        proteins_present_by_spec[spec].append((name_orig,None,y1000_id,swissprot_id, source))
            
                        prot_seq = selected_proteins_seqs[name_orig]
                        header = '>' + header_raw + '\n'  
                        

                    f_out_prot.write(header)
                    f_out_prot.write(prot_seq + '\n')

                    f_out_filt.write(header)
                    f_out_filt.write(str(record.seq) + '\n')
                    



# Save proteins_present_by_spec

#remove_duplicates
for spec in specs: 
    proteins_present_by_spec[spec] = list(set(proteins_present_by_spec[spec]))

with open(proteins_present_by_spec_fname, 'w') as f:
    json.dump(proteins_present_by_spec, f, sort_keys=True, indent=4 )


In [81]:
#Identifies source of coding sequences for each species
cds_params = { 
                 'saccharomyces_cerevisiae': {'af2': genomes_dir + os.sep + os.path.normpath('saccharomyces_cerevisiae/S288C_reference_genome_R64-2-1_20150113/orf_coding_all_R64-2-1_20150113.fasta') },
                 'schizosaccharomyces_pombe': {'af2': genomes_dir + os.sep + os.path.normpath('schizosaccharomyces_pombe/cds.fa')},
                 'candida_albicans' : {'af2': genomes_dir + os.sep + os.path.normpath('candida_albicans/C_albicans_SC5314_A22_current_default_coding.fasta')}
             }
             
    
#shen_only                 
shen_specs = [
                 'kazachstania_naganishii',
                 'geotrichum_candidum',
                 'vanderwaltozyma_polyspora',                              
                 'eremothecium_gossypii',
                 'ascoidea_rubescens',
                 'cyberlindnera_jadinii',
                 'ogataea_parapolymorpha',
                 'pachysolen_tannophilus',
                 'yHMPu5000034604_sporopachydermia_lactativora_160519',
                 'alloascoidea_hylecoeti',
                 'candida_apicola',
                 'yarrowia_lipolytica',
                 'tortispora_caseinolytica',
                 'lipomyces_starkeyi',
                 #when/uniprot-ENA
                 'wickerhamomyces_anomalus',
                 'yHMPu5000034957_hanseniaspora_osmophila_160519',
                 'candida_tropicalis',
                 #shen/uniprot-NCBI
                 'zygosaccharomyces_rouxii',
                 'debaryomyces_hansenii',
                 'lachancea_thermotolerans',
                 'kluyveromyces_marxianus',
                 'kluyveromyces_lactis',
                 'komagataella_pastoris',
                 'torulaspora_delbrueckii'
                  ]

for spec in shen_specs: 
    cds_params[spec] = {'shen':y1000plus_dir + os.sep + os.path.normpath('shen_2018_data/0_332yeast_genomes/332_genome_annotations/cds/' + spec +'.max.cds')}

#uniprot-ENA
uniprot_ena_specs = [
                 'wickerhamomyces_anomalus',
                 'yHMPu5000034957_hanseniaspora_osmophila_160519',
                 'candida_tropicalis'
                  ]

for spec in uniprot_ena_specs: 
    cds_params[spec]['uniprot_ena'] = genomes_dir + os.sep + os.path.normpath('diverse_strains/ena/' + spec + '/cds.fasta')

    
#uniprot-NCBI
uniprot_ncbi_specs = [
                 'zygosaccharomyces_rouxii',
                 'debaryomyces_hansenii',
                 'lachancea_thermotolerans',
                 'kluyveromyces_marxianus',
                 'kluyveromyces_lactis',
                 'komagataella_pastoris'
                     ]

for spec in uniprot_ncbi_specs: 
    cds_params[spec]['uniprot_ncbi'] = genomes_dir + os.sep + os.path.normpath('diverse_strains/ncbi_coding_seq/'+ spec + '/cds.fasta')
                 
#     }

In [17]:
#Load proteins_present_by_spec

proteins_present_by_spec_fname = base_dir + os.sep + 'selected_protein_ids.json'
    
with open(proteins_present_by_spec_fname, 'r') as f:
    proteins_present_by_spec = json.load(f) 

In [30]:
#proteins_present_by_spec['kluyveromyces_lactis']
#['REF_Scer_AF-P16661-F1-model_v2', 'YBR110W', '110_477', 'P16661']

In [ ]:
#load cds file
#load peptide file
#Extract appropriate proteins from cds file
#print into a new cds file with protein name from file

In [82]:
for cds_source in ['af2', 'shen','uniprot_ncbi', 'uniprot_ena']: 
    print(cds_source)
    for spec, cds_params_spec in cds_params.items():
        print(spec)
        if cds_source in cds_params_spec.keys():
            cds_out_fname = base_dir + os.sep + os.path.normpath('selected_proteins/cds/' + spec+'__'+ cds_source + '.fasta')
            #Load CDS file
            cds_fasta = SeqIO.parse(cds_params_spec[cds_source], 'fasta')
            
            #make dictionary keyed on ID of cds seq
            cds_fasta_dict = {}
            for record in cds_fasta:
                cds_fasta_dict[record.id] = str(record.seq)
 
            #for shen genomes, make lookup for y1000_id
            #Load y1000_id gene_full lookup
            y1000_id_table = pd.read_csv(y1000plus_dir + os.sep + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/' + spec + '.csv'))
            y1000_id_2_gene_full = dict(zip(y1000_id_table['y1000_id'],y1000_id_table['gene_full']))            

            proteins_present = proteins_present_by_spec[spec]
            
            with open(cds_out_fname,'w') as f_cds_out: 
                for (name_orig,gene_id,y1000_id,uniprot_id,source) in proteins_present:
                    if cds_source=='af2': 
                        try:
                            f_cds_out.write('>' + name_orig+ ' source=' + source + ' source_id=' + gene_id + '\n')
                            f_cds_out.write(cds_fasta_dict[gene_id] + '\n')
                        except KeyError:
                            print(gene_id + 'not found in ' + spec + ' ' + cds_source + ' cds file')
                    
                    if cds_source=='shen': 
                        if source=='shen':
                            try:
                                gene_full_cds = y1000_id_2_gene_full[y1000_id] + '-mRNA-1'
                                f_cds_out.write('>' + name_orig+ ' source=' + source + ' source_id=' + gene_full_cds + '\n')
                                f_cds_out.write(cds_fasta_dict[gene_full_cds] + '\n')
                            except KeyError:
                                print(gene_full +  ' not found in ' + spec + ' ' + cds_source + ' cds file')
                    
                    if cds_source=='uniprot_ena':
                        if source=='uniprot':
                            try:
                                f_cds_out.write('>' + name_orig + ' source=' + source + ' source_id=' + uniprot_id + '\n')
                                f_cds_out.write(cds_fasta_dict[uniprot_id] + '\n')
                            except KeyError:
                                print(gene_full +  ' not found in ' + spec + ' ' + cds_source + ' cds file')

                    if cds_source=='uniprot_ncbi':
                        if source=='uniprot':
                            try:
                                f_cds_out.write('>' + name_orig + ' source=' + source + ' source_id=' + uniprot_id + '\n')
                                f_cds_out.write(cds_fasta_dict[uniprot_id] + '\n')
                            except KeyError:
                                print(gene_full +  ' not found in ' + spec + ' ' + cds_source + ' cds file')
    

af2
saccharomyces_cerevisiae
schizosaccharomyces_pombe
candida_albicans
kazachstania_naganishii
geotrichum_candidum
vanderwaltozyma_polyspora
eremothecium_gossypii
ascoidea_rubescens
cyberlindnera_jadinii
ogataea_parapolymorpha
pachysolen_tannophilus
yHMPu5000034604_sporopachydermia_lactativora_160519
alloascoidea_hylecoeti
candida_apicola
yarrowia_lipolytica
tortispora_caseinolytica
lipomyces_starkeyi
wickerhamomyces_anomalus
yHMPu5000034957_hanseniaspora_osmophila_160519
candida_tropicalis
zygosaccharomyces_rouxii
debaryomyces_hansenii
lachancea_thermotolerans
kluyveromyces_marxianus
kluyveromyces_lactis
komagataella_pastoris
torulaspora_delbrueckii
shen
saccharomyces_cerevisiae
schizosaccharomyces_pombe
candida_albicans
kazachstania_naganishii
geotrichum_candidum
vanderwaltozyma_polyspora
eremothecium_gossypii
ascoidea_rubescens
cyberlindnera_jadinii
ogataea_parapolymorpha
pachysolen_tannophilus
yHMPu5000034604_sporopachydermia_lactativora_160519
alloascoidea_hylecoeti
candida_apico

In [88]:
#Build a big dictionary for selected proteins and for selected proteins

#Proteins is made up of selected_proteins.fasta and selected_proteins_modelorgs.fasta
#
#CDS is made up of all files in 
#G:\My Drive\Crick_LMS\projects\diverse_yeasts\alphafold\selected_proteins\cds

# #Load original sequence file, and make dictionary of gene_id to fasta header and gene_id to peptide sequence
# #Read in Sequence File, extract all names make dict of seq_alignment sequences
# selected_proteins = SeqIO.parse(base_dir +os.sep +  'selected_proteins.fasta', 'fasta')
# #selected_proteins_headers = {}
# selected_proteins_seqs = {}
# for record in selected_proteins: 
#     #selected_proteins_headers[record.id] = record.description
#     selected_proteins_seqs[record.id] = str(record.seq)
    
# all_selected_proteins_seqs = selected_proteins_seqs.copy()

# #Load selected_proteins for  model species, include in dictionary
# model_selected_proteins = SeqIO.parse(base_dir +os.sep +  'selected_proteins_modelorgs.fasta', 'fasta')

# for record in model_selected_proteins: 
#     record_metadata = record.description.split(' ')[1:]

#     metadata_dict = {}
#     for metadata_line in record_metadata: 
#         metadata_label, metadata = metadata_line.split('=')
#         metadata_dict[metadata_label]=metadata
        
#     all_selected_proteins_seqs[metadata_dict['af2_id']] = str(record.seq)

#Make a dictionary of coding sequences
cds_dir = base_dir + os.sep + os.path.normpath('selected_proteins/cds') + os.sep

selected_cds_seqs = {}

for cds_file in os.listdir(cds_dir):
    cds_fasta = SeqIO.parse(cds_dir + cds_file, 'fasta')
    for record in cds_fasta: 
        selected_cds_seqs[record.id] = str(record.seq)

#Iterate through each msa and make a codon and protein sequence fasta (unaligned)

seq_dir = base_dir + os.sep + 'og_sequences'

for protein_file in os.listdir(seq_dir + os.sep + 'proteome' ):
    cds_file_out = seq_dir + os.sep + 'cds' + os.sep + protein_file.split('.')[0] + '.cds.fasta'
    
    protein_file_full = seq_dir + os.sep+ 'proteome' + os.sep + protein_file    
    protein_file_fasta = SeqIO.parse(protein_file_full, 'fasta')
    
    with open(cds_file_out,'w') as f_out_cds: 
        for record in protein_file_fasta: 
            f_out_cds.write(record.id + '\n')
            f_out_cds.write(selected_cds_seqs[record.id] + '\n')
            
    
    
    
    
    




In [30]:
#Model species info

#C. albicans
#http://www.candidagenome.org/download/sequence/C_albicans_SC5314/Assembly22/current/  
# C_albicans_SC5314_A22_current_default_coding.fasta
# C_albicans_SC5314_A22_current_default_protein.fasta
#downloaded on 20221012 and stored in
#G:\My Drive\Crick_LMS\external_data\genomes\Candida_albicans


#S. cerevisiae
#G:\My Drive\Crick_LMS\external_data\genomes\Saccharomyces_cerevisiae\S288C_reference_genome_R64-2-1_20150113
#orf_coding_all_R64-2-1_20150113.fasta
#orf_trans_all_R64-2-1_20150113.fasta

#S. pombe
#From https://www.pombase.org/data/genome_sequence_and_features/feature_sequences/ on 20221012
#cds.fa 
#peptide.fa  
#both last modified 2022-10-12 03:26
#Stored at
#G:\My Drive\Crick_LMS\external_data\genomes\Schizosaccharomyces_pombe

## Make a Fasta of structures for S.pom, S.cer, and C. alb that we need Kcat/Km from

In [ ]:
#Import og_metadata